# Layers and Blocks

To begin, we revisit the code
that we used to implement MLPs

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 2.2297e-02,  4.8330e-03,  6.2722e-02,  2.0200e-02,  1.2944e-01,
         -1.5532e-01, -2.9793e-01,  1.3160e-01, -6.8483e-02, -4.1429e-02],
        [ 6.6891e-02, -5.0645e-05, -5.6356e-03, -1.5112e-01,  2.5051e-01,
         -1.5891e-01, -2.3494e-01,  1.7370e-01,  1.9335e-01,  2.9268e-03]],
       grad_fn=<AddmmBackward>)

`nn.Sequential` defines a special kind of `Module`

A Custom Block

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

Instantiate the MLP's layers
and subsequently invoke these layers

In [3]:
net = MLP()
net(X)

tensor([[-0.0147, -0.1417,  0.0125,  0.0428, -0.0649, -0.0065,  0.0477, -0.1715,
          0.2559, -0.1557],
        [-0.0136,  0.0603, -0.1399, -0.0411,  0.1092, -0.0738,  0.0248, -0.0855,
          0.1849, -0.0956]], grad_fn=<AddmmBackward>)

The Sequential Block

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0052, -0.0552,  0.1817,  0.1063, -0.1553, -0.0345,  0.0183, -0.0564,
         -0.0268,  0.1044],
        [ 0.0626,  0.0601,  0.1270, -0.0450, -0.2169, -0.1182,  0.0572, -0.0006,
          0.0140, -0.0165]], grad_fn=<AddmmBackward>)

Executing Code in the Forward Propagation Function

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.2863, grad_fn=<SumBackward0>)

Mix and match various
ways of assembling blocks together

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.0655, grad_fn=<SumBackward0>)